In [1]:
library(tidyverse)
library(RPostgreSQL)
library(RPostgreSQL)
library(twang)
library(Matching)
library(tidyverse)
library(survey)
library(tidyverse)
library(foreign)
library(MASS)
library(readxl)
library(tableone)
library(VIM)
library(mice)
library(reshape)
library(tidyr)
### RCS ####
library(ggplot2)
library(rms)

Warning message:
"package 'tidyverse' was built under R version 4.3.3"
Warning message:
"package 'ggplot2' was built under R version 4.3.3"
Warning message:
"package 'readr' was built under R version 4.3.3"
Warning message:
"package 'forcats' was built under R version 4.3.3"
Warning message:
"package 'lubridate' was built under R version 4.3.3"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"package 'RPostgreSQL' was built under R version 4.3.3"
Loading required package: DBI

Warning message:
"package

In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:

con <- dbConnect(RPostgres::Postgres(),
                  host = "127.0.0.1",
                  port = 5432,
                  dbname = "mimiciv",
                  user = "postgres",
                  password = "123")
full_data = dbGetQuery(con, "select*from mimiciv_hosp.merged_data_gcs")


In [4]:
colnames(full_data)

[1] "stay_id"                        "hadm_id"                       
  [3] "subject_id"                     "gender"                        
  [5] "age"                            "sapsii"                        
  [7] "icu_intime"                     "icu_outtime"                   
  [9] "weight"                         "sofa_24hours"                  
 [11] "infection"                      "mort_28_day"                   
 [13] "survival_days"                  "gcs_include"                   
 [15] "gcs_include_combine"            "vs_heart_rate_first"           
 [17] "vs_heart_rate_min"              "vs_heart_rate_max"             
 [19] "vs_cvp_first"                   "vs_cvp_min"                    
 [21] "vs_cvp_max"                     "vs_map_first"                  
 [23] "vs_map_min"                     "vs_map_max"                    
 [25] "vs_temp_first"                  "vs_temp_min"                   
 [27] "vs_temp_max"                    "lab_hemoglobin_first"          
 [29] "lab_hemoglobin_min"             "lab_hemoglobin_max"            
 [31] "lab_hemoglobin_abnormal"        "lab_platelet_first"            
 [33] "lab_platelet_min"               "lab_platelet_max"              
 [35] "lab_platelet_abnormal"          "lab_creatinine_kinase_first"   
 [37] "lab_creatinine_kinase_min"      "lab_creatinine_kinase_max"     
 [39] "lab_creatinine_kinase_abnormal" "lab_wbc_first"                 
 [41] "lab_wbc_min"                    "lab_wbc_max"                   
 [43] "lab_wbc_abnormal"               "lab_ph_first"                  
 [45] "lab_ph_min"                     "lab_ph_max"                    
 [47] "lab_ph_abnormal"                "lab_chloride_first"            
 [49] "lab_chloride_min"               "lab_chloride_max"              
 [51] "lab_chloride_abnormal"          "lab_sodium_first"              
 [53] "lab_sodium_min"                 "lab_sodium_max"                
 [55] "lab_sodium_abnormal"            "lab_bun_first"                 
 [57] "lab_bun_min"                    "lab_bun_max"                   
 [59] "lab_bun_abnormal"               "lab_bicarbonate_first"         
 [61] "lab_bicarbonate_min"            "lab_bicarbonate_max"           
 [63] "lab_bicarbonate_abnormal"       "lab_pco2_first"                
 [65] "lab_pco2_min"                   "lab_pco2_max"                  
 [67] "lab_pco2_abnormal"              "lab_creatinine_first"          
 [69] "lab_creatinine_min"             "lab_creatinine_max"            
 [71] "lab_creatinine_abnormal"        "lab_potassium_first"           
 [73] "lab_potassium_min"              "lab_potassium_max"             
 [75] "lab_potassium_abnormal"         "lab_troponin_first"            
 [77] "lab_troponin_min"               "lab_troponin_max"              
 [79] "lab_troponin_abnormal"          "lab_po2_first"                 
 [81] "lab_po2_min"                    "lab_po2_max"                   
 [83] "lab_po2_abnormal"               "lab_lactate_first"             
 [85] "lab_lactate_min"                "lab_lactate_max"               
 [87] "lab_lactate_abnormal"           "pre_dose_val_rx"               
 [89] "los_drug"                       "cumulative_dose"               
 [91] "gcs"                            "ca"                            
 [93] "infection_flag"                 "dose_val_rx_average"           
 [95] "los_drug_average"               "event_state"                   
 [97] "vs_heart_rate_flag"             "vs_cvp_flag"                   
 [99] "vs_map_flag"                    "vs_temp_flag"                  
[101] "lab_hemoglobin_flag"            "lab_platelet_flag"             
[103] "lab_creatinine_kinase_flag"     "lab_wbc_flag"                  
[105] "lab_ph_flag"                    "lab_chloride_flag"             
[107] "lab_sodium_flag"                "lab_bun_flag"                  
[109] "lab_bicarbonate_flag"           "lab_bnp_flag"                  
[111] "lab_pco2_flag"                  "la

In [5]:
to_factor <- function(x) {
    if(length(na.omit(unique(x))) <= 1) return(factor(x, levels = c(0, 1)))
    return(factor(x))
}

In [6]:

factor_vars <- full_data %>%
    names %>%
    grep("flag|abnormal|gcs", ., value = TRUE) %>%
    c("gender",  "mort_28_day", "event_state", "gcs", "ca","gcs_include_combine")
factor_vars

[1] "gcs_include"                    "gcs_include_combine"           
 [3] "lab_hemoglobin_abnormal"        "lab_platelet_abnormal"         
 [5] "lab_creatinine_kinase_abnormal" "lab_wbc_abnormal"              
 [7] "lab_ph_abnormal"                "lab_chloride_abnormal"         
 [9] "lab_sodium_abnormal"            "lab_bun_abnormal"              
[11] "lab_bicarbonate_abnormal"       "lab_pco2_abnormal"             
[13] "lab_creatinine_abnormal"        "lab_potassium_abnormal"        
[15] "lab_troponin_abnormal"          "lab_po2_abnormal"              
[17] "lab_lactate_abnormal"           "gcs"                           
[19] "infection_flag"                 "vs_heart_rate_flag"            
[21] "vs_cvp_flag"                    "vs_map_flag"                   
[23] "vs_temp_flag"                   "lab_hemoglobin_flag"           
[25] "lab_platelet_flag"              "lab_creatinine_kinase_flag"    
[27] "lab_wbc_flag"                   "lab_ph_flag"                   
[29] "lab_chloride_flag"              "lab_sodium_flag"               
[31] "lab_bun_flag"                   "lab_bicarbonate_flag"          
[33] "lab_bnp_flag"                   "lab_pco2_flag"                 
[35] "lab_creatinine_flag"            "lab_potassium_flag"            
[37] "lab_troponin_flag"              "lab_po2_flag"                  
[39] "lab_lactate_flag"               "gender"                        
[41] "mort_28_day"                    "event_state"                   
[43] "gcs"                            "ca"                            
[45] "gcs_include_combine"

In [7]:
full_data <- full_data %>%
  mutate(gcs_include_combine_int = as.integer(gcs_include_combine)) %>%
  mutate(ca_int = as.integer(ca)) %>%
  mutate(event_state_int = as.integer(event_state)) %>%
  mutate(mort_28_day_int = as.integer(mort_28_day)) %>%
  mutate_at(factor_vars, to_factor) %>%
  mutate(gender = relevel(gender, "M")) 

In [8]:
feature_names <- full_data %>%
    names %>%
    keep(grepl("vs|lab|age|gender|weight|sapsii|sofa|sofa_24hours|event_state|cam_state|gcs", .)) %>%
    discard(grepl("vs|lab", .) & grepl("flag", .) & !grepl("bnp|troponin|kinase|cvp", .)) %>%

    discard(grepl("min|max|flag|gcs_include", .)) %>%
    discard(grepl("abnormal", .))
feature_names
length(feature_names)

[1] "gender"                      "age"                        
 [3] "sapsii"                      "weight"                     
 [5] "sofa_24hours"                "vs_heart_rate_first"        
 [7] "vs_cvp_first"                "vs_map_first"               
 [9] "vs_temp_first"               "lab_hemoglobin_first"       
[11] "lab_platelet_first"          "lab_creatinine_kinase_first"
[13] "lab_wbc_first"               "lab_ph_first"               
[15] "lab_chloride_first"          "lab_sodium_first"           
[17] "lab_bun_first"               "lab_bicarbonate_first"      
[19] "lab_pco2_first"              "lab_creatinine_first"       
[21] "lab_potassium_first"         "lab_troponin_first"         
[23] "lab_po2_first"               "lab_lactate_first"          
[25] "gcs"                         "dose_val_rx_average"        
[27] "los_drug_average"            "event_state"                
[29] "event_state_int"

[1] 29

In [9]:

feature_names %>%
    data.frame(feature = ., stringsAsFactors = FALSE) %>%
    data.table::fwrite(file = file.path(data_dir, "feature_names3.csv"))

In [10]:
features <- full_data %>%
    dplyr::select(!!!rlang::syms(feature_names))
head(features)

,gender,age,sapsii,weight,sofa_24hours,vs_heart_rate_first,vs_cvp_first,vs_map_first,vs_temp_first,lab_hemoglobin_first,⋯,lab_creatinine_first,lab_potassium_first,lab_troponin_first,lab_po2_first,lab_lactate_first,gcs,dose_val_rx_average,los_drug_average,event_state,event_state_int
,<fct>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>,<dbl>,<fct>,<int>
1,F,66.04835,31,138.0,1,143,NA,78,38.27778,8.5,⋯,0.7,3.9,NA,40,1.1,0,NA,NA,1,1
2,F,41.91122,39,70.0,0,140,NA,99,39.44444,8.0,⋯,0.5,3.0,NA,NA,NA,1,50,2,0,0
3,F,87.59137,35,65.0,1,97,NA,110,36.72222,11.4,⋯,2.2,3.7,NA,40,2.7,0,5,136,0,0
4,M,66.88656,28,88.0,3,72,NA,82,37.05556,13.5,⋯,1.1,4.1,NA,NA,NA,0,NA,NA,0,0
5,F,44.39155,55,45.9,10,110,16,78,37.33333,13.0,⋯,1.4,2.5,NA,296,2.4,0,NA,NA,1,1
6,M,56.61848,30,80.0,3,110,4,86,NA,11.4,⋯,1.0,4.3,NA,223,1.0,0,NA,NA,0,0


In [11]:
label_name <- "gcs_include"
label <- full_data %>% pull(gcs_include)
str(label)

 Factor w/ 2 levels "0","1": 1 2 1 1 1 1 2 1 1 1 ...


In [12]:
label <- full_data %>% pull(gcs)
str(label)

 Factor w/ 2 levels "0","1": 1 2 1 1 1 1 2 1 1 1 ...


In [13]:

fml <- "mort_28_day ~ gcs_include + gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first +  lab_po2_first + lab_lactate_first"

In [14]:
unweighted <- glm(as.formula(fml), data = full_data, family = binomial, na.action = na.exclude)
summary(unweighted)


Call:
glm(formula = as.formula(fml), family = binomial, data = full_data, 
    na.action = na.exclude)

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -4.9834123  9.8553248  -0.506 0.613098    
gcs_include1          -0.0404799  0.2292858  -0.177 0.859864    
genderF                0.3942072  0.1631194   2.417 0.015663 *  
age                    0.0208507  0.0061461   3.393 0.000693 ***
sapsii                 0.0330349  0.0061967   5.331 9.76e-08 ***
weight                -0.0137895  0.0039460  -3.495 0.000475 ***
sofa_24hours           0.2162730  0.0247158   8.750  < 2e-16 ***
vs_heart_rate_first    0.0074115  0.0037301   1.987 0.046928 *  
vs_map_first          -0.0002146  0.0038383  -0.056 0.955418    
vs_temp_first         -0.1044461  0.0807706  -1.293 0.195969    
lab_hemoglobin_first  -0.0386125  0.0372900  -1.035 0.300452    
lab_platelet_first     0.0004273  0.0006381   0.670 0.503116    
lab_wbc_first          0.0031915  0.

In [15]:

exp(cbind(OR = coef(unweighted), confint(unweighted)))

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.006850646,1.921037e-11,1.173442e+06
gcs_include1,0.960328470,6.048227e-01,1.488672e+00
genderF,1.483207909,1.078862e+00,2.046237e+00
age,1.021069616,1.008956e+00,1.033579e+00
sapsii,1.033586655,1.021089e+00,1.046220e+00
weight,0.986305111,9.785687e-01,9.938306e-01
sofa_24hours,1.241441265,1.183413e+00,1.303950e+00
vs_heart_rate_first,1.007439033,1.000089e+00,1.014833e+00
vs_map_first,0.999785443,9.922002e-01,1.007236e+00
vs_temp_first,0.900823340,7.710793e-01,1.057658e+00


In [16]:
saveRDS(unweighted, file = file.path(data_dir, "multivariate_model_combine.rds"))

In [17]:
fml <-  "gcs_include_int ~ gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_cvp_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_creatinine_kinase_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first + lab_troponin_first + lab_po2_first + lab_lactate_first"

In [18]:
full_data$pre_dose_val_rx <- as.numeric(full_data$pre_dose_val_rx)
full_data$pre_dose_val_rx[is.na(full_data$pre_dose_val_rx)] <- 0

In [19]:
full_data$pre_dose_val_rx_group <- cut(full_data$pre_dose_val_rx, 
                                       breaks = c(min(full_data$pre_dose_val_rx), 12.44, 53.73, max(full_data$pre_dose_val_rx)),
                                       labels = c("Low", "Medium", "High"))

In [20]:
full_data$pre_dose_val_rx_group <- as.character(full_data$pre_dose_val_rx_group)

# Replace NA values in pre_dose_val_rx_group with 0
full_data$pre_dose_val_rx_group[is.na(full_data$pre_dose_val_rx_group)] <- "0"

full_data$pre_dose_val_rx_group

[1] "0"      "Medium" "0"      "0"      "0"      "0"      "Medium" "0"     
   [9] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
  [17] "0"      "0"      "0"      "0"      "0"      "0"      "High"   "0"     
  [25] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "Low"   
  [33] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
  [41] "0"      "0"      "0"      "0"      "0"      "Low"    "0"      "0"     
  [49] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
  [57] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
  [65] "0"      "0"      "0"      "Low"    "0"      "High"   "0"      "0"     
  [73] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
  [81] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
  [89] "Low"    "0"      "0"      "0"      "0"      "0"      "0"      "0"     
  [97] "0"      "0"      "Low"    "0"      "0"      "0"      "0"      "0"     
 [105] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [113] "0"      "0"      "0"      "Low"    "0"      "0"      "0"      "0"     
 [121] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "Low"   
 [129] "0"      "0"      "0"      "Low"    "0"      "0"      "0"      "Low"   
 [137] "0"      "0"      "0"      "0"      "0"      "Low"    "0"      "0"     
 [145] "0"      "0"      "0"      "0"      "Low"    "0"      "0"      "0"     
 [153] "0"      "Low"    "0"      "0"      "0"      "0"      "0"      "0"     
 [161] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [169] "0"      "0"      "0"      "0"      "0"      "Medium" "0"      "0"     
 [177] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [185] "Medium" "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [193] "Low"    "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [201] "0"      "Low"    "0"      "0"      "Medium" "0"      "0"      "0"     
 [209] "0"      "0"      "0"      "0"      "Low"    "0"      "0"      "0"     
 [217] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [225] "0"      "0"      "0"      "Medium" "0"      "0"      "0"      "0"     
 [233] "0"      "0"      "Low"    "0"      "Medium" "0"      "0"      "Low"   
 [241] "0"      "High"   "0"      "0"      "0"      "Low"    "0"      "0"     
 [249] "0"      "0"      "0"      "Medium" "0"      "0"      "0"      "0"     
 [257] "0"      "0"      "0"      "0"      "0"      "High"   "0"      "0"     
 [265] "0"      "Low"    "0"      "0"      "0"      "Low"    "0"      "0"     
 [273] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "Low"   
 [281] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [289] "0"      "Low"    "0"      "0"      "0"      "0"      "0"      "0"     
 [297] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [305] "0"      "0"      "0"      "Medium" "0"      "0"      "0"      "0"     
 [313] "0"      "Low"    "0"      "0"      "0"      "0"      "0"      "0"     
 [321] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [329] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [337] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [345] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [353] "Low"    "0"      "Medium" "0"      "0"      "0"      "0"      "0"     
 [361] "0"      "0"      "0"      "0"      "0"      "0"      "0"      "Low"   
 [369] "0"      "0"      "High"   "0"      "0"      "0"      "0"      "0"     
 [377] "Low"    "0"      "0"      "0"      "0"      "0"      "0"      "0"     
 [385] "0"      "0"      "0"      "0"      "Medium" "0"      "0"      "0"     
 [393] "0"      "0"      "0"      "0"      "0"      "High"   "0"      "0"     
 [401] "0"      "0"      "0"      "0"      "Low"    "

In [21]:
features <- c("age", "gender",  "weight","pre_dose_val_rx_group",
              "sapsii", "sofa_24hours", "mort_28_day", "survival_days", "infection_falg", 
              "vs_map_first", "vs_heart_rate_first", "vs_temp_first", 
              "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
              "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
              "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
              "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
              "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

              

In [22]:
summary(full_data)


    stay_id            hadm_id           subject_id       gender  
 Min.   :30002548   Min.   :20002810   Min.   :10001217   M:2084  
 1st Qu.:32431171   1st Qu.:22479831   1st Qu.:12613157   F:2317  
 Median :34987065   Median :24912349   Median :15063388           
 Mean   :34963598   Mean   :24989187   Mean   :15058878           
 3rd Qu.:37443929   3rd Qu.:27547009   3rd Qu.:17528049           
 Max.   :39999172   Max.   :29995505   Max.   :19999987           
                                                                  
      age             sapsii         icu_intime                    
 Min.   : 18.16   Min.   :  2.00   Min.   :2110-01-12 00:54:00.00  
 1st Qu.: 54.30   1st Qu.: 27.00   1st Qu.:2134-10-22 07:41:31.00  
 Median : 64.51   Median : 35.00   Median :2154-09-25 13:12:24.00  
 Mean   : 64.11   Mean   : 36.85   Mean   :2154-10-26 14:18:01.78  
 3rd Qu.: 75.22   3rd Qu.: 44.00   3rd Qu.:2174-07-16 21:25:07.00  
 Max.   :100.52   Max.   :105.00   Max.   :2211-05-01 06

In [23]:
tab <- CreateTableOne(vars = features,
                      strata = "pre_dose_val_rx_group",
                      data = full_data,
                      argsNormal = list(var.equal = FALSE))


Warning message in ModuleReturnVarsExist(vars, data):
"The data frame does not have: infection_falg  Dropped"


In [24]:
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               dplyr::select(-test)) %>% invisible

In [25]:
tab_df

,0,High,Low,Medium,p,SMD
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
n,3868,54,328,151,,
age (mean (SD)),64.02 (15.15),62.18 (15.96),66.41 (14.56),62.11 (15.21),0.010,0.163
gender = F (%),2016 ( 52.1),34 ( 63.0),188 ( 57.3),79 ( 52.3),0.132,0.127
weight (mean (SD)),81.05 (22.06),86.02 (24.17),78.49 (21.35),82.52 (26.07),0.069,0.172
pre_dose_val_rx_group (%),,,,,<0.001,NaN
0,3868 (100.0),0 ( 0.0),0 ( 0.0),0 ( 0.0),,
High,0 ( 0.0),54 (100.0),0 ( 0.0),0 ( 0.0),,
Low,0 ( 0.0),0 ( 0.0),328 (100.0),0 ( 0.0),,
Medium,0 ( 0.0),0 ( 0.0),0 ( 0.0),151 (100.0),,


In [66]:
library(broom)
fit1 <- glm(mort_28_day ~ pre_dose_val_rx_group + age + gender + gender + weight, 
            family = binomial(link = "logit"), 
            data = full_data)
pooled1 <- tidy(fit1)
pooled1


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-3.148047143,0.376260322,-8.36667318,5.926779e-17
pre_dose_val_rx_groupHigh,-0.213718909,0.529555288,-0.40358186,6.865202e-01
pre_dose_val_rx_groupLow,-0.029748753,0.194290023,-0.15311519,8.783074e-01
pre_dose_val_rx_groupMedium,0.703118654,0.231373226,3.03889376,2.374486e-03
age,0.030423613,0.003690718,8.24327715,1.675568e-16
genderF,0.007463732,0.109606975,0.06809541,9.457097e-01
weight,-0.014446069,0.002844316,-5.07892541,3.795757e-07


In [84]:
#利用tableone包的ShowRegTable函数产生OR值和置信区间
library(tableone)
table1<-ShowRegTable(fit1, exp = TRUE, digits = 3, pDigits = 3,
                      printToggle = TRUE, quote = FALSE, ciFun = confint) 

                            exp(coef) [confint]  p     
(Intercept)                 0.043 [0.020, 0.089] <0.001
pre_dose_val_rx_groupHigh   0.808 [0.241, 2.026]  0.687
pre_dose_val_rx_groupLow    0.971 [0.652, 1.401]  0.878
pre_dose_val_rx_groupMedium 2.020 [1.258, 3.127]  0.002
age                         1.031 [1.024, 1.038] <0.001
genderF                     1.007 [0.813, 1.250]  0.946
weight                      0.986 [0.980, 0.991] <0.001


In [63]:

fit1 <- exp(cbind(OR = coef(fit1), confint(fit1)))


Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.04293589,0.02037526,0.08909589
pre_dose_val_rx_groupHigh,0.80757535,0.24056421,2.02636885
pre_dose_val_rx_groupLow,0.97068939,0.65240875,1.40051950
pre_dose_val_rx_groupMedium,2.02004271,1.25802246,3.12734516
age,1.03089114,1.02353920,1.03845971
genderF,1.00749165,0.81295043,1.24959378
weight,0.98565777,0.98010120,0.99109106


In [85]:
# Extract the covariate names from the model object
covariate_names <- names(coef(fit1))

# Create a data frame with the covariate names and the table3 output
table1_df <- data.frame(Covariate = covariate_names, table1)

# Write the data frame to a CSV file with column names
data.table::fwrite(table1_df, file.path(data_dir, "table1.csv"), col.names = TRUE)

In [28]:
library(broom)
fit2 <- glm(mort_28_day ~ pre_dose_val_rx_group + age + gender + weight + 
          sapsii + sofa_24hours + lab_wbc_first + lab_hemoglobin_first + lab_platelet_first, 
            family = binomial(link = "logit"), 
            data = full_data)
pooled1 <- tidy(fit2)
pooled1


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-4.9964236273,0.5137041380,-9.726267043,2.329865e-22
pre_dose_val_rx_groupHigh,-0.3171675333,0.5455935192,-0.581325698,5.610210e-01
pre_dose_val_rx_groupLow,-0.0011127749,0.2098550095,-0.005302589,9.957692e-01
pre_dose_val_rx_groupMedium,0.5856298284,0.2511209031,2.332063246,1.969736e-02
age,0.0223642715,0.0042874812,5.216179459,1.826512e-07
genderF,0.1769829353,0.1196650872,1.478985555,1.391442e-01
weight,-0.0182126425,0.0030848138,-5.903968275,3.548604e-09
sapsii,0.0399399536,0.0044648522,8.945414426,3.705567e-19
sofa_24hours,0.1893627821,0.0190669501,9.931466829,3.037538e-23


In [68]:
exp(cbind(OR = coef(fit2), confint(fit2)))

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.006762088,0.002445944,0.01833679
pre_dose_val_rx_groupHigh,0.728208746,0.211911090,1.89960327
pre_dose_val_rx_groupLow,0.998887844,0.651770975,1.48690266
pre_dose_val_rx_groupMedium,1.796121878,1.077711271,2.89351916
age,1.022616227,1.014132467,1.03132771
genderF,1.193610724,0.944608770,1.51040083
weight,0.981952205,0.975949588,0.98782474
sapsii,1.040748279,1.031675739,1.04990272
sofa_24hours,1.208479288,1.164374645,1.25480480
lab_wbc_first,1.003334153,0.994348962,1.01107819


In [81]:
#利用tableone包的ShowRegTable函数产生OR值和置信区间
library(tableone)
table2<-ShowRegTable(fit2, exp = TRUE, digits = 3, pDigits = 3,
                      printToggle = TRUE, quote = FALSE, ciFun = confint) 

                            exp(coef) [confint]  p     
(Intercept)                 0.007 [0.002, 0.018] <0.001
pre_dose_val_rx_groupHigh   0.728 [0.212, 1.900]  0.561
pre_dose_val_rx_groupLow    0.999 [0.652, 1.487]  0.996
pre_dose_val_rx_groupMedium 1.796 [1.078, 2.894]  0.020
age                         1.023 [1.014, 1.031] <0.001
genderF                     1.194 [0.945, 1.510]  0.139
weight                      0.982 [0.976, 0.988] <0.001
sapsii                      1.041 [1.032, 1.050] <0.001
sofa_24hours                1.208 [1.164, 1.255] <0.001
lab_wbc_first               1.003 [0.994, 1.011]  0.413
lab_hemoglobin_first        0.980 [0.929, 1.033]  0.448
lab_platelet_first          1.001 [1.000, 1.002]  0.039


In [83]:
# Extract the covariate names from the model object
covariate_names <- names(coef(fit2))

# Create a data frame with the covariate names and the table3 output
table2_df <- data.frame(Covariate = covariate_names, table2)

# Write the data frame to a CSV file with column names
data.table::fwrite(table2_df, file.path(data_dir, "table2.csv"), col.names = TRUE)

In [30]:
library(broom)
fit3 <- glm(mort_28_day ~ pre_dose_val_rx_group + gender + age + sapsii + weight + sofa_24hours + vs_heart_rate_first + vs_map_first + vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + lab_potassium_first + lab_po2_first + lab_lactate_first,
            family = binomial(link = "logit"), 
            data = full_data)
pooled1 <- tidy(fit3)
pooled1

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-5.177488e+00,9.8510774464,-0.525575783,5.991830e-01
pre_dose_val_rx_groupHigh,-7.412800e-01,0.7043112616,-1.052489190,2.925752e-01
pre_dose_val_rx_groupLow,1.141289e-03,0.2935658433,0.003887675,9.968981e-01
pre_dose_val_rx_groupMedium,1.100468e-01,0.3657318813,0.300894758,7.634947e-01
genderF,4.050009e-01,0.1635906389,2.475697381,1.329762e-02
age,2.082811e-02,0.0061543621,3.384283637,7.136424e-04
sapsii,3.321668e-02,0.0062217326,5.338815459,9.355581e-08
weight,-1.369394e-02,0.0039505710,-3.466320427,5.276341e-04
sofa_24hours,2.165969e-01,0.0247881220,8.737932890,2.374145e-18


In [70]:
exp(cbind(OR = coef(fit3), confint(fit3)))

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.005642163,1.590979e-11,9.587155e+05
pre_dose_val_rx_groupHigh,0.476503605,9.823155e-02,1.662508e+00
pre_dose_val_rx_groupLow,1.001141940,5.494207e-01,1.743888e+00
pre_dose_val_rx_groupMedium,1.116330320,5.236516e-01,2.216106e+00
genderF,1.499303874,1.089632e+00,2.070491e+00
age,1.021046526,1.008918e+00,1.033573e+00
sapsii,1.033774515,1.021227e+00,1.046464e+00
weight,0.986399391,9.786528e-01,9.939335e-01
sofa_24hours,1.241843472,1.183631e+00,1.304561e+00
vs_heart_rate_first,1.007388780,1.000032e+00,1.014791e+00


In [78]:
#利用tableone包的ShowRegTable函数产生OR值和置信区间
library(tableone)
table3<-ShowRegTable(fit3, exp = TRUE, digits = 3, pDigits = 3,
                      printToggle = TRUE, quote = FALSE, ciFun = confint) 
data.table::fwrite(table3, file.path(data_dir, "table3.csv"), col.names = TRUE)


                            exp(coef) [confint]       p     
(Intercept)                 0.006 [0.000, 958715.489]  0.599
pre_dose_val_rx_groupHigh   0.477 [0.098, 1.663]       0.293
pre_dose_val_rx_groupLow    1.001 [0.549, 1.744]       0.997
pre_dose_val_rx_groupMedium 1.116 [0.524, 2.216]       0.763
genderF                     1.499 [1.090, 2.070]       0.013
age                         1.021 [1.009, 1.034]       0.001
sapsii                      1.034 [1.021, 1.046]      <0.001
weight                      0.986 [0.979, 0.994]       0.001
sofa_24hours                1.242 [1.184, 1.305]      <0.001
vs_heart_rate_first         1.007 [1.000, 1.015]       0.049
vs_map_first                1.000 [0.992, 1.007]       0.981
vs_temp_first               0.901 [0.771, 1.058]       0.197
lab_hemoglobin_first        0.963 [0.894, 1.036]       0.308
lab_platelet_first          1.000 [0.999, 1.002]       0.543
lab_wbc_first               1.003 [0.989, 1.017]       0.669
lab_ph_first            

In [80]:
# Extract the covariate names from the model object
covariate_names <- names(coef(fit3))

# Create a data frame with the covariate names and the table3 output
table3_df <- data.frame(Covariate = covariate_names, table3)

# Write the data frame to a CSV file with column names
data.table::fwrite(table3_df, file.path(data_dir, "table3.csv"), col.names = TRUE)